In [5]:
#-----USE THIS SCRIPT TO ADD A LIST OF SPECIES TO THE DATABASE 

from Bio import Entrez
from Bio import SeqIO
from time import sleep  #---this command is optional 
import numpy as np

import os

#----------CHANGE WORKING DIRECTORY TO YOUR WORKING FOLDER

os.chdir('your\\directory') 


Entrez.email="your.email@gmail.com" #-----ENTER YOUR EMAIL ADDRESS

ft=open("file_for_taxonomy", "a") #----TAXONOMY OUTPUT FILE, new taxonomy will be added 

fs=open("file_for_sequences.txt", "a") #-----REFERENCE SEQUENCES OUTPUT FILE, new sequences will be added

i=1
e=1

                
#----SEARCH NCBI 16S RefSeq DATABASE FOR THE LISTED SPECIES
with open("your\\directory\\bacterial_species.txt", "r") as ifile:
        for line in ifile:
            search_phrase=line.replace('\n', '').replace('\r', '') + "[organism] AND 33175[BioProject]"
            handle = Entrez.esearch(db="nucleotide", term=search_phrase, retmax=10)
            result = Entrez.read(handle) 
            handle.close()
            if not result["IdList"]:
                print ('Did not find %s.' %(search_phrase))
                search_phrase=line.replace('\n', '').replace('\r', '') + "[organism] AND 33317[BioProject]"
                handle = Entrez.esearch(db="nucleotide", term=search_phrase, retmax=10)
                result = Entrez.read(handle) 
                handle.close()  
                if not result["IdList"]:
                    print ('Did not find %s.' %(search_phrase))
                    search_phrase=line.replace('\n', '').replace('\r', '') + "[organism] AND 16S[ALL FIELDS] AND RNA[All FIELDS]"
                    handle = Entrez.esearch(db="nucleotide", term=search_phrase, retmax=10)
                    result = Entrez.read(handle) 
                    handle.close() 
                    if not result["IdList"]:
                        print ('Also Did not find %s. Omitting this line' %(search_phrase))
                        sleep (0.5)
                        continue
                    
                       
            print ('Search phrase: %s' %(search_phrase))
            fetch_handle = Entrez.efetch(db="nuccore", id=result["IdList"], rettype="acc", retmode="text")
            acc_ids = [id.strip() for id in fetch_handle]
            my_acc_array=np.asarray(acc_ids)
            
            fetch_handle.close()

            seq_len = 0
            my_seq_array = []
            for SId in my_acc_array:
                handle = Entrez.efetch(db="nucleotide", id=SId, rettype="gb", retmode="fasta")
                record = SeqIO.read(handle, "genbank")
                handle.close()
                
#-------SELECT THE LONGEST SEQUENCE FROM ALL FOUNDS GENBANK RECORDS 
                sequence = record.seq 
                if len(sequence) > seq_len:
                    seq_len = len(sequence)
                    seq_max = sequence
                    r=record
                    sleep(0.5)
                    continue
                    
                
#------EXTRACT TAXONOMY
                   
            try:
                kingdom=r.annotations["taxonomy"][0]
            except IndexError:
                kingdom=" "                    
            try:
                phylum=r.annotations["taxonomy"][1]
            except IndexError:
                phylum=" "
            try:
                clas=r.annotations["taxonomy"][2]
            except IndexError:
                clas=" "
            try:
                order=r.annotations["taxonomy"][3]
            except IndexError:
                order=" "
            try:
                family=r.annotations["taxonomy"][4]
            except IndexError:
                family=" "
            try:
                genus=r.annotations["taxonomy"][5]
            except IndexError:
                genus=" "
            try:
                source=r.annotations["organism"]
            except IndexError:
                source=" "
            out_file_taxonomy=SId+"\tk__"+kingdom+"; p__"+phylum+"; c__"+clas+"; o__"+order+"; f__"+family+"; g__"+genus+"; s__"+source+"\n"
            ft.write(out_file_taxonomy)
            i+=1
            
#--------EXTRACT REFERENCE SEQUENCES IN TEXT FORMAT
                        
            out_file_sequence=">"+SId+"\n"+str(seq_max)+"\n"
            fs.write(out_file_sequence)
            e+=1
            sleep(5.0)
            continue
            
#-------DOWNLOAD REFERENCE SEQUENCES IN FASTA FORMAT

            out_file=SId+'.fasta'
            sleep (0.5) 
            SeqIO.write(r, out_file, "fasta")
                        
ft.close()
          
fs.close()


print("done!")
                    
                    





Search phrase: citrobacter freundii[organism] AND 33175[BioProject]
done!
